In [2]:
import os

In [3]:
cd ..

d:\Full Stack Data Science\Time Series Analysis\MAJOR PROJECT\SMDF


In [4]:
%pwd

'd:\\Full Stack Data Science\\Time Series Analysis\\MAJOR PROJECT\\SMDF'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    M01AB: Path
    M01AE: Path
    N02BA: Path
    N02BE: Path
    N05B: Path
    N05C: Path
    R03: Path
    R06: Path
    scaler: Path
    

In [6]:
from SMDF.constants import *
from SMDF.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation

        create_directories([config.root_dir])

        model_eval_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            M01AB = config.M01AB,
            M01AE = config.M01AE,
            N02BA = config.N02BA,
            N02BE = config.N02BE,
            N05B = config.N05B,
            N05C = config.N05C,
            R03 = config.R03,
            R06 = config.R06,
            scaler = config.scaler,
            

            
        )

        return model_eval_config

In [8]:

import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import numpy as np

# Deep learning
from sklearn.preprocessing import StandardScaler
from SMDF.logging import logger
import joblib



In [1]:
import tensorflow as tf

In [32]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config


    def noramalizing_data(self,column_name):

        scaler = StandardScaler()
        train = pd.read_csv(self.config.train_data_path)
        test = pd.read_csv(self.config.test_data_path)
        train.drop(columns=["datum","Year","Month","Hour","Weekday Name"],inplace=True)
        test.drop(columns=["datum","Year","Month","Hour","Weekday Name"],inplace=True)

        
        
        #train[columns] = train[columns]
        #logger.info(train.head())
        train_data = scaler.fit_transform(train[column_name].values.reshape(-1,1))

        test_data = scaler.transform(test[column_name].values.reshape(-1,1))

        
        logger.info(test_data.shape)
        
        #print(test_data)
        return  test_data,train_data


    
    def eval_metrics(self,actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    
    def test_spliting(self,train, test):
        window_size = 60
        # Concatenate train data to test data
        dataset_total = np.concatenate((train, test), axis = 0)
        # Split test data and last window-size of train data
        inputs = dataset_total[len(dataset_total) - len(test) - window_size:]
        # Do the same thing for test data
        X_test = []
        y_test = []
        for i in range(window_size, window_size+len(test)):
            X_test.append(inputs[i-window_size:i,:]) 
            y_test.append(inputs[i,-1]) # consider Close as target
        # Change them to numpy array
        X_test, y_test = np.array(X_test).astype('float32'), np.array(y_test).reshape(-1, 1)
        logger.info(X_test.shape)
    
        return X_test,y_test
    
    def eval(self,X_test,y_test,key):
        
        columns = [ self.config.M01AB,
            self.config.M01AE,
            self.config.N02BA,
            self.config.N02BE,
            self.config.N05B,
            self.config.N05C,
            self.config.R03,
            self.config.R06]
        time_distribution = ['time_distributed_1',
                             'time_distributed_3',
                             'time_distributed_5',
                             'time_distributed_7',
                             'time_distributed_9',
                             'time_distributed_11',
                             'time_distributed_13',
                             'time_distributed_15']
        
        loaded_model = tf.saved_model.load(columns[key])
        model = loaded_model.signatures["serving_default"]
        test_predict = model(tf.constant(X_test))
        #print(test_predict)
        predictions = test_predict[time_distribution[key]].numpy().squeeze()
        
        #test_predict = test_predict.reshape(-1,1)
        #prin
        rmse, mae, r2 = self.eval_metrics(y_test.squeeze(),predictions)
        logger.info(f"No.{key} of RMSE : {rmse}, MAE: {mae}, r2: {r2} ")
        return predictions
        """
        test_predict = test_predict_5.reshape(-1)  # Reshape to 1-dimensional array
        print('RMSE:', np.sqrt(mean_squared_error(y_test, test_predict_5)))

        plt.figure(figsize=(20, 10))
        plt.grid(b=True, which='major', axis='both')
        plt.title('Testing prediction result with Hybrid Model', fontsize=30)
        plt.plot(test_data.index, y_test, linewidth=4, label='True', color='blue')
        plt.plot(test_data.index, test_predict_5, linewidth=4, color='red', label='Predicted')
        plt.xticks(fontsize=20)
        plt.yticks(fontsize=20)
        plt.legend(fontsize=25, shadow=True, facecolor='cyan', edgecolor='k')
        plt.show()

        """


In [33]:
try:
    config = ConfigurationManager()
    config = config.get_model_evaluation_config()
    eval = ModelEvaluation(config=config)
    
    columns = ["M01AB",
            "M01AE",
            "N02BA",
            "N02BE",
            "N05B",
            "N05C",
            "R03",
            "R06"]
    
    for key,val in enumerate(columns):
        train,test = eval.noramalizing_data(val)
        X_test, y_test = eval.test_spliting(train,test)
        logger.info(f"Evaluation Metrics of {val}")
        eval.eval(X_test,y_test,key)

    
        

except Exception as e:
    raise e

[2023-07-02 09:32:03,383: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-07-02 09:32:03,386: INFO: common: yaml file: params.yaml loaded successfully]
[2023-07-02 09:32:03,391: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-07-02 09:32:03,393: INFO: common: created directory at: artifacts]
[2023-07-02 09:32:03,395: INFO: common: created directory at: artifacts/model_evaluation]
[2023-07-02 09:32:03,408: INFO: 324198258: (422, 1)]
[2023-07-02 09:32:03,413: INFO: 324198258: (1684, 60, 1)]
[2023-07-02 09:32:03,414: INFO: 1008040765: Evaluation Metrics of M01AB]


[2023-07-02 09:32:06,235: INFO: 324198258: No.0 of RMSE : 0.1729812573821065, MAE: 0.044478520548772854, r2: 0.9700774845945054 ]
[2023-07-02 09:32:06,252: INFO: 324198258: (422, 1)]
[2023-07-02 09:32:06,259: INFO: 324198258: (1684, 60, 1)]
[2023-07-02 09:32:06,261: INFO: 1008040765: Evaluation Metrics of M01AE]
[2023-07-02 09:32:07,806: INFO: 324198258: No.1 of RMSE : 0.22746234897838546, MAE: 0.07061078990499645, r2: 0.9482608797972352 ]
[2023-07-02 09:32:07,825: INFO: 324198258: (422, 1)]
[2023-07-02 09:32:07,830: INFO: 324198258: (1684, 60, 1)]
[2023-07-02 09:32:07,832: INFO: 1008040765: Evaluation Metrics of N02BA]
[2023-07-02 09:32:09,360: INFO: 324198258: No.2 of RMSE : 0.2960777272525262, MAE: 0.08451347883882948, r2: 0.9123379794249787 ]
[2023-07-02 09:32:09,385: INFO: 324198258: (422, 1)]
[2023-07-02 09:32:09,395: INFO: 324198258: (1684, 60, 1)]
[2023-07-02 09:32:09,397: INFO: 1008040765: Evaluation Metrics of N02BE]
[2023-07-02 09:32:11,012: INFO: 324198258: No.3 of RMSE : 0

In [20]:
config.train_data_path

'artifacts/data_spliting/train.csv'